In [ ]:
import weave
from weave import weave_internal
from weave.old_weave.ecosystem.all import langchain
weave.use_frontend_devmode()

In [ ]:
weave.old_weave.panels.Board(
    vars={
        "chat_model": langchain.chat_openai("gpt-3.5-turbo", 0.7),
        "animals": weave.old_weave.panels.Query(
            ["koala", "giraffe", "leopard"],
            text=lambda item: item
        ),
        "objects": weave.old_weave.panels.Query(
            ["spoon", "guitar"],
            text=lambda item: item
        ),
        "prompts": weave.old_weave.panels.Query([
                "A {animal} with an {object}",
                "A 20-word story about a {animal} with an {object}",
                "You are creative writer. Describe a scene with a {animal} and a {object}. Choose an art style and use adjectives that evoke it."
            ],
            text=lambda item: item),
        "combos": lambda animals, objects, prompts:
            weave.ops.cross_product(weave.ops.dict_(
                animal=animals.pinned_data(),
                object=objects.pinned_data(),
                prompt=prompts.pinned_data())),

    },
    panels=[
        weave.old_weave.panels.BoardPanel(
            lambda chat_model, combos: weave.old_weave.panels.Facet(
                combos,
                cell_size=(256, 256),
                x=lambda item: item['animal'] + ' ' + item['object'],
                y=lambda item: item['prompt'],
                select=lambda items: 
                weave.old_weave.panels.Group(
                    vars={
                        'gen_text': chat_model.predict(
                            items[0]['prompt'].format(weave.ops.dict_(
                                animal=items[0]['animal'],
                                object=items[0]['object'],
                            ))),
                    },
                    items={
                        'text': lambda gen_text: gen_text,
                        'image': lambda gen_text: gen_text.stable_diffusion()
                    }
                )
            ),
            layout=weave.old_weave.panels.BoardPanelLayout(x=0, y=0, w=24, h=24)
        )
#         weave.old_weave.panels.BoardPanel(
#             lambda col_names: weave.old_weave.panels.Table(col_names, columns=[lambda col: col]),
#             layout=weave.old_weave.panels.BoardPanelLayout(x=0, y=12, w=12, h=6)
#         ),
#         weave.old_weave.panels.BoardPanel(
#             lambda row_names: weave.old_weave.panels.Table(row_names, columns=[lambda row: row]),
#             layout=weave.old_weave.panels.BoardPanelLayout(x=12, y=12, w=12, h=6)
#         ),
    ]
)